# Amazon SageMaker Notebook Instance + SSH + IDE Integration

Run this notebook inside a [SageMaker Notebook instance](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi.html). For [SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html) use [SageMaker_SSH_IDE.ipynb](SageMaker_SSH_IDE.ipynb) instead.

In [ ]:
%%sh
pip install -U pip
pip uninstall --root-user-action ignore -y awscli
pip install --root-user-action ignore -q sagemaker-ssh-helper

In [ ]:
from sagemaker_ssh_helper.wrapper import SSHEstimatorWrapper
from sagemaker.pytorch import PyTorch
import sagemaker
import logging
import os

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
!mkdir -p ssh_notebook/

In [ ]:
%%writefile ssh_notebook/ssh_notebook.py

import time
import os

import sagemaker_ssh_helper
sagemaker_ssh_helper.setup_and_start_ssh()

while os.environ.get("START_SSH", "false") == "true":
    time.sleep(10)  # will sleep forever

In the next cell you need to put your **local** user id. It will help to tag the managed instance with your id as the owner. You can find the id by running `aws sts get-caller-identity` command on your local machine. The value will be returned as `UserId`.

In [ ]:
%%sh
LOCAL_USER_ID="AIDACKCEVSQ6C2EXAMPLE"  # replace with your local UserId

if [ -f ~/.sm-ssh-owner ]; then
    echo "sm-ssh-notebook: SSH owner user ID is already set in ~/.sm-ssh-owner, skipping override"
else
    echo "sm-ssh-notebook: Saving SSH owner user ID into ~/.sm-ssh-owner"
    echo "$LOCAL_USER_ID" > ~/.sm-ssh-owner
fi

In [ ]:
with open(os.path.expanduser('~/.sm-ssh-owner')) as f:
    local_user_id = f.readline().rstrip()

Now start the estimator with SSH wrapper. It will connect to the SSM and start SSH server inside the container.

Keep the cell running.

On the local machne, open the SSH tunnel to the notebook with `sm-local-ssh-notebook connect <<<notebook-instance-name>>`.

The estimator will mirror the current notebook directory to `/opt/ml/input/data/notebook` inside the container. You can use it to configure the mappings in your local IDE, so that your local files are synchronized with notebook instance files. See [the instructions for PyCharm](https://www.jetbrains.com/help/pycharm/deployment-mappings-tab.html), as an example.

In [ ]:
estimator = PyTorch(entry_point='ssh_notebook.py',
                    source_dir='ssh_notebook/',
                    dependencies=[SSHEstimatorWrapper.dependency_dir()],
                    base_job_name='ssh-notebook',
                    role=sagemaker.get_execution_role(),
                    framework_version='1.9.1',
                    py_version='py38',
                    instance_count=1,
                    instance_type='local',
                    container_log_level=logging.INFO)

ssh_wrapper = SSHEstimatorWrapper.create(estimator, connection_wait_time_seconds=0, local_user_id=local_user_id, log_to_stdout=True)

estimator.fit({'notebook': 'file://'})